In [ ]:
import requests
import pandas as pd
import io
import time
import datetime
import numpy as np
import json

In [ ]:
TWSE_list = pd.DataFrame()
def Crawler_TWSE_List():
  global TWSE_list
  url = 'https://isin.twse.com.tw/isin/C_public.jsp?strMode=2'
  page = requests.get(url)
  TWSE_list = pd.read_html(page.text)[0]
  TWSE_list.columns = TWSE_list.iloc[0]
  end = 0
  for i in range(0,len(TWSE_list)):
    if (TWSE_list.at[i, '有價證券代號及名稱'][0:4] == '上市認購'):
        end = i
  TWSE_list = TWSE_list.iloc[2:end]
  TWSE_list = TWSE_list[['有價證券代號及名稱', '國際證券辨識號碼(ISIN Code)']]
  TWSE_list  = TWSE_list.rename(columns={'有價證券代號及名稱': 'Stockcode', '國際證券辨識號碼(ISIN Code)': 'Name'})
  TWSE_list.reset_index(drop=True, inplace=True)
  for i in range (0,len(TWSE_list)):
    TWSE_list.at[i, 'Name'] = TWSE_list.at[i, 'Stockcode'][5:]
    TWSE_list.at[i, 'Stockcode'] = TWSE_list.at[i, 'Stockcode'][0:4]

In [ ]:
TPEx_list = pd.DataFrame()
def Crawler_TPEx_List():
  global TPEx_list
  url = 'https://isin.twse.com.tw/isin/C_public.jsp?strMode=4'
  page = requests.get(url)
  TPEx_list = pd.read_html(page.text)[0]
  TPEx_list.columns = TPEx_list.iloc[0]
  start = 0
  end = 0
  for i in range(0,len(TPEx_list)):
    if (TPEx_list.at[i, '有價證券代號及名稱'][0:4] == '股票'):
        start = i+1
    if (TPEx_list.at[i, '有價證券代號及名稱'][0:4] == '特別股'):
        end = i
  TPEx_list = TPEx_list.iloc[start:end]
  TPEx_list = TPEx_list[['有價證券代號及名稱', '國際證券辨識號碼(ISIN Code)']]
  TPEx_list = TPEx_list.rename(columns={'有價證券代號及名稱': 'Stockcode', '國際證券辨識號碼(ISIN Code)': 'Name'})
  TPEx_list.reset_index(drop=True, inplace=True)
  for i in range (0,len(TPEx_list)):
    TPEx_list.at[i, 'Name'] = TPEx_list.at[i, 'Stockcode'][5:]
    TPEx_list.at[i, 'Stockcode'] = TPEx_list.at[i, 'Stockcode'][0:4]

In [8]:
StockPrice_TWSE = pd.read_excel('Stock_TWSE.xlsx', header=[0,1], index_col=0)
StockPrice_TPEx = pd.read_excel('Stock_TPEx.xlsx', header=[0,1], index_col=0)

In [17]:
StockPrice_TWSE = pd.read_csv('Stock_TWSE.csv', header=[0,1], index_col=0)
StockPrice_TPEx = pd.read_csv('Stock_TPEx.csv', header=[0,1], index_col=0)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc1 in position 16: invalid start byte

In [ ]:
different_Tpex = None
different_TWSE = None
def Update_Stock_List():
  global different_Tpex
  global different_TWSE
  Crawler_TPEx_List()
  Crawler_TWSE_List()
  if list(dict.fromkeys(StockPrice_TPEx.columns.get_level_values(0)[1:])) == list(TPEx_list.iloc[:,0]):
    print('TPEx Pass')
  else:
    if (len(TPEx_list) > (len(dict.fromkeys(StockPrice_TPEx.columns.get_level_values(0)[1:]))-1)):
        different_Tpex = set(TPEx_list.iloc[:,0]) - set(StockPrice_TPEx.columns.get_level_values(0)[1:])
        different_Tpex = list(different_Tpex)
        print(different_Tpex)
        for i in range(0,len(different_Tpex)):
            StockPrice_TPEx.insert(len(StockPrice_TPEx.columns), (different_Tpex[i], 'Close'), None)
            StockPrice_TPEx.insert(len(StockPrice_TPEx.columns), (different_Tpex[i], 'Volume'), None)
            StockPrice_TPEx.insert(len(StockPrice_TPEx.columns), (different_Tpex[i], 'Foundation'), None)
            StockPrice_TPEx.insert(len(StockPrice_TPEx.columns), (different_Tpex[i], 'Yield'), None)
            StockPrice_TPEx.sort_index(axis = 1, level=0, inplace=True, sort_remaining = False)
            temp = StockPrice_TPEx.pop(('Date',       'Date'))
            StockPrice_TPEx.insert(0, ('Date',       'Date'), temp)
    else:
        different_Tpex = set(StockPrice_TPEx.columns.get_level_values(0)[1:]) - set(TPEx_list.iloc[:,0])
        different_Tpex = list(different_Tpex)
        print(different_Tpex)
        for i in range(0,len(different_Tpex)):
            StockPrice_TPEx.drop(columns=[different_Tpex[i]], inplace=True)
            
  if list(dict.fromkeys(StockPrice_TWSE.columns.get_level_values(0)[1:])) == list(TWSE_list.iloc[:,0]):
    print('TWSE Pass')
  else:
    if (len(TWSE_list) > (len(dict.fromkeys(StockPrice_TWSE.columns.get_level_values(0)[1:]))-1)):
        different_TWSE = set(TWSE_list.iloc[:,0]) - set(StockPrice_TWSE.columns.get_level_values(0)[1:])
        different_TWSE = list(different_TWSE)
        print(different_TWSE)
        for i in range(0,len(different_TWSE)):
            StockPrice_TWSE.insert(len(StockPrice_TWSE.columns), (different_TWSE[i], 'Close'), None)
            StockPrice_TWSE.insert(len(StockPrice_TWSE.columns), (different_TWSE[i], 'Volume'), None)
            StockPrice_TWSE.insert(len(StockPrice_TWSE.columns), (different_TWSE[i], 'Foundation'), None)
            StockPrice_TWSE.insert(len(StockPrice_TWSE.columns), (different_TWSE[i], 'Yield'), None)
            StockPrice_TWSE.sort_index(axis = 1, level=0, inplace=True, sort_remaining = False)
            temp = StockPrice_TWSE.pop(('Date',       'Date'))
            StockPrice_TWSE.insert(0, ('Date',       'Date'), temp)
    else:
        different_TWSE = set(StockPrice_TWSE.columns.get_level_values(0)[1:]) - set(TWSE_list.iloc[:,0])
        different_TWSE = list(different_TWSE)
        print(different_TWSE)
        for i in range(0,len(different_TWSE)):
            StockPrice_TWSE.drop(columns=[different_TWSE[i]], inplace=True)

In [ ]:
def crawler(datetime):
    url = 'https://www.tpex.org.tw/web/stock/aftertrading/otc_quotes_no1430/stk_wn1430_result.php?l=zh-tw&o=csv&d='+ datetime +'&se=EW&s=0,asc,0'
    page = requests.get(url)
    use_text = page.text.splitlines()
    for i,text in enumerate(use_text):
        if '代號' in text:
            initial_point = i
            break
    test_df = pd.read_csv(io.StringIO(''.join([text+ '\n' for text in use_text[initial_point:]])))
    for i in range(0,len(test_df)):
        try:
            if (len(test_df.loc[i,'代號']) != 4):
                test_df.drop(i,inplace=True)
        except TypeError:
            if test_df.loc[i,'代號'] < 1200 and test_df.loc[i,'代號'] > 9999:
                test_df.drop(i,inplace=True)
    test_df = test_df[['代號', '收盤 ', '成交股數  ']]
    return test_df

def crawler_Foundation(datetime):
    url = 'https://www.tpex.org.tw/web/stock/3insti/daily_trade/3itrade_hedge_result.php?l=zh-tw&o=csv&se=EW&t=D&d='+ datetime +'&s=0,asc'
    page = requests.get(url)
    use_text = page.text.splitlines()
    for i,text in enumerate(use_text):
        if '代號' in text:
            initial_point = i
            break
    test_df = pd.read_csv(io.StringIO(''.join([text+ '\n' for text in use_text[initial_point:]])))
    if len(test_df) > 1:
        for i in range(0,len(test_df)):
            try:
                if (len(test_df.loc[i,'代號']) != 4):
                    test_df.drop(i,inplace=True)
            except TypeError:
                if test_df.loc[i,'代號'] < 1200 and test_df.loc[i,'代號'] > 9999:
                    test_df.drop(i,inplace=True)
        try:
            test_df = test_df[['代號', '三大法人買賣超股數合計']]
        except KeyError:
            test_df = test_df[['代號', '三大法人買賣超股數']]
        test_df.reset_index(drop=True,inplace=False)
    else:
        del test_df
    return test_df

def crawler_Yield(datetime):
    url = 'https://www.tpex.org.tw/web/stock/aftertrading/peratio_analysis/pera_result.php?l=zh-tw&o=csv&charset=UTF-8&d='+datetime+'&c=&s=0,asc'
    page = requests.get(url)
    use_text = page.text.splitlines()
    for i,text in enumerate(use_text):
            if '股票代號' in text:
                initial_point = i
                break
    test_df = pd.read_csv(io.StringIO(''.join([text+ '\n' for text in use_text[initial_point:]])))
    if (len(use_text) < 10):
        del test_df
    else:
        test_df = test_df[['股票代號', '殖利率(%)']]
        test_df = test_df.rename(columns={"股票代號": '代號'})
    return test_df

#Day_TPEx = []
def trans_date(datetime):
#    global Day_TPEx
    d = datetime.day
    m = datetime.month
    y = datetime.year-1911
    if m < 10:
        if d < 10:
            Day_TPEx = (str(y)+'/0'+str(m)+'/0'+str(d))
        else:
            Day_TPEx = (str(y)+'/0'+str(m)+'/'+str(d))
    else:
        if d < 10:
            Day_TPEx = (str(y)+'/'+str(m)+'/0'+str(d))
        else:
            Day_TPEx = (str(y)+'/'+str(m)+'/'+str(d))
            
    return Day_TPEx

Crawler_Price = pd.DataFrame()
Crawler_Foundation = pd.DataFrame()
Crawler_Yield = pd.DataFrame()
def crawler_data(datetime):
    try:
        global Crawler_Price
        global Crawler_Foundation
        global Crawler_Yield
        Crawler_Price = crawler(trans_date(datetime))
        Crawler_Foundation = crawler_Foundation(trans_date(datetime))
        Crawler_Yield = crawler_Yield(trans_date(datetime))
        print('TPEx Crawler Successful')
    except UnboundLocalError:
        print('TPEx Crawler Fail')
    except KeyError:
        print('TPEx Crawler Fail')

In [ ]:
def catch(datetime):
    url = 'https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date='+datetime+'&type=ALLBUT0999&response=json'
    page = requests.get(url).text
    json_data = json.loads(page)
    Stock_data = json_data['tables'][8]['data']
    test_df = pd.DataFrame(Stock_data, columns = [json_data['tables'][8]['fields']])
    for i in range(0,len(test_df)):
        if ((len(test_df.loc[i,'證券代號'][0])) != 4):
            test_df.drop(i,inplace=True)
        elif int(test_df.loc[i,'證券代號'][0]) < 1100 or int(test_df.loc[i,'證券代號'][0]) > 9999:
            test_df.drop(i,inplace=True)
    temp = test_df[['證券代號', '收盤價', '成交股數']]
    test_df = pd.DataFrame(temp.values, columns = [temp.columns[0][0],temp.columns[1][0],temp.columns[2][0]])
    return test_df

def catch_Foundation(datetime):
    url = 'https://www.twse.com.tw/rwd/zh/fund/T86?date='+datetime+'&selectType=ALLBUT0999&response=json'
    page = requests.get(url).text
    json_data = json.loads(page)
    Stock_data = json_data['data']
    test_df = pd.DataFrame(Stock_data, columns = [json_data['fields']])
    for i in range(0,len(test_df)):
        if ((len(test_df.loc[i,'證券代號'][0])) != 4):
            test_df.drop(i,inplace=True)
        elif int(test_df.loc[i,'證券代號'][0]) < 1100 or int(test_df.loc[i,'證券代號'][0]) > 9999:
            test_df.drop(i,inplace=True)
    temp = test_df[['證券代號', '三大法人買賣超股數']]
    test_df = pd.DataFrame(temp.values, columns = [temp.columns[0][0],temp.columns[1][0]])
    test_df.sort_values(by=['證券代號'], inplace=True)
    test_df.reset_index(drop=True,inplace=True)
    return test_df

def catch_Yield(datetime):
    url = 'https://www.twse.com.tw/rwd/zh/afterTrading/BWIBBU_d?date='+datetime+'&selectType=ALL&response=json'
    page = requests.get(url).text
    json_data = json.loads(page)
    Stock_data = json_data['data']
    test_df = pd.DataFrame(Stock_data, columns = [json_data['fields']])
    temp = test_df[['證券代號', '殖利率(%)']]
    test_df = pd.DataFrame(temp.values, columns = [temp.columns[0][0],temp.columns[1][0]])
    return test_df
    
def catch_day(datetime):
    d = datetime.day
    m = datetime.month
    y = datetime.year
    if m < 10:
        if d < 10:
            Day_TWSE = str(y)+'0'+str(m)+'0'+str(d)
        else:
            Day_TWSE = str(y)+'0'+str(m)+str(d)
    else:
        if d < 10:
            Day_TWSE = str(y)+str(m)+'0'+str(d)
        else:
            Day_TWSE = str(y)+str(m)+str(d)
            
    return Day_TWSE

Catch_Price = pd.DataFrame()
Catch_Foundation = pd.DataFrame()
Catch_Yield = pd.DataFrame()
def catch_data(datetime):
    try:
        global Catch_Price
        global Catch_Foundation
        global Catch_Yield
        Catch_Price = catch(catch_day(datetime))
        Catch_Foundation = catch_Foundation(catch_day(datetime))
        Catch_Yield = catch_Yield(catch_day(datetime))
        print('TWSE Crawler Successful')
    except UnboundLocalError:
        print('TWSE Crawler Fail')
    except KeyError:
        print('TWSE Crawler Fail')

In [ ]:
Crawler_Frame = pd.DataFrame()
Catch_Frame = pd.DataFrame()
def Get_New_Data(datetime):
    start = 0
    global Crawler_Frame
    global Crawler_Price
    global Crawler_Foundation
    global Crawler_Yield
    global Catch_Frame
    global Catch_Price
    global Catch_Foundation
    global Catch_Yield
    crawler_data(datetime)
    Crawler_Frame = pd.concat([Crawler_Price.set_index('代號'), Crawler_Foundation.set_index('代號').reindex(Crawler_Price.set_index('代號').index), Crawler_Yield.set_index('代號').reindex(Crawler_Price.set_index('代號').index)], axis=1)
    Crawler_Frame = Crawler_Frame.reset_index()
    catch_data(datetime)
    Catch_Frame = pd.concat([Catch_Price.set_index('證券代號'), Catch_Foundation.set_index('證券代號').reindex(Catch_Price.set_index('證券代號').index), Catch_Yield.set_index('證券代號').reindex(Catch_Price.set_index('證券代號').index)], axis=1)
    Catch_Frame = Catch_Frame.reset_index()

In [ ]:
def Time_transfer(datetime):
  d = datetime.day
  m = datetime.month
  y = datetime.year
  if m < 10:
    if d < 10:
      Time = str(y)+'-0'+str(m)+'-0'+str(d)
    else:
      Time = str(y)+'-0'+str(m)+'-'+str(d)
  else:
    if d < 10:
      Time = str(y)+'-'+str(m)+'-0'+str(d)
    else:
      Time = str(y)+'-'+str(m)+'-'+str(d)
  return Time

In [ ]:
def Update_Stock_Price(datetime):
  temp = 0
  try:
    url = 'https://www.tpex.org.tw/web/stock/aftertrading/otc_quotes_no1430/stk_wn1430_result.php?l=zh-tw&o=csv&d='+trans_date(datetime)+'&se=EW&s=0,asc,0'
    page = requests.get(url)
    use_text = page.text.splitlines()
    for i,text in enumerate(use_text):
      if '代號' in text:
        initial_point = i
        break
    test_df = pd.read_csv(io.StringIO(''.join([text+ '\n' for text in use_text[initial_point:]])))
  except NameError:
    print('No New Trading Data')
    temp = 1
    
#  global different_Tpex
#  global different_TWSE
#  Update_Stock_List()
                            
  if temp == 0:
    Update_Stock_List()
    Get_New_Data(datetime)
    global StockPrice_TWSE
    global StockPrice_TPEx
    temp1 = 0
    temp2 = 0
    if StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,0] == Time_transfer(datetime):
      print('TWSE Exists!')
      temp1 = 1
    else:
      StockPrice_TWSE = StockPrice_TWSE.append({('Date',       'Date'):Time_transfer(datetime)}, ignore_index=True)
    
    if StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,0] == Time_transfer(datetime):
      print('TPEx Exists!')
      temp2 = 1
    else:
      StockPrice_TPEx = StockPrice_TPEx.append({('Date',       'Date'):Time_transfer(datetime)}, ignore_index=True)
    
    if temp2 == 0:
      for i in range (0,len(Crawler_Frame)):
        for j in range (1, len(list(StockPrice_TPEx)),4):
          if (Crawler_Frame.loc[i,'代號'] == list(StockPrice_TPEx)[j][0]):
            try:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j] = float(Crawler_Frame.iloc[i,1])
            except ValueError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j] = None
            
            try:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+1] = float(Crawler_Frame.iloc[i,2].replace(',',''))
            except ValueError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+1] = None
            except AttributeError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+1] = None
            
            try:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+2] = float(Crawler_Frame.iloc[i,3].replace(',',''))
            except ValueError:              
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+2] = None
            except AttributeError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+1] = None
            
            try:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+3] = float(Crawler_Frame.iloc[i,4])
            except ValueError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+3] = None
    
    if temp1 == 0:
      for x in range (0,len(Catch_Frame)):
        for y in range (1, len(list(StockPrice_TWSE)),4):
          if (Catch_Frame.loc[x,'證券代號'] == list(StockPrice_TWSE)[y][0]):
            try:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y] = float(Catch_Frame.iloc[x,1])
            except ValueError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y] = None
            
            try:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+1] = float(Catch_Frame.iloc[x,2].replace(',',''))
            except ValueError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+1] = None
            except AttributeError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+1] = None
            
            try:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+2] = float(Catch_Frame.iloc[x,3].replace(',',''))
            except ValueError:              
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+2] = None
            except AttributeError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+1] = None
            
            try:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+3] = float(Catch_Frame.iloc[x,4])
            except ValueError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+3] = None
            
    print('Finished!')  

In [ ]:
Update_Stock_Price(datetime.datetime(2023, 3, 30, 10, 1, 55, 280233))

In [ ]:
StockPrice_TWSE

In [ ]:
StockPrice_TPEx

In [16]:
StockPrice_TWSE.to_csv('Stock_TWSE.csv')
StockPrice_TPEx.to_csv('Stock_TPEx.csv')

In [ ]:
MA_TWSE = pd.DataFrame(index=['7MA', '14MA', '30MA', '60MA', '90MA', '180MA', '360MA', '720MA', '1080MA'],columns=StockPrice_TWSE.columns.get_level_values(0)[1:].drop_duplicates())
MA_TWSE.index.name ='Moving Average'
MA_TPEx = pd.DataFrame(index=['7MA', '14MA', '30MA', '60MA', '90MA', '180MA', '360MA', '720MA', '1080MA'],columns=StockPrice_TPEx.columns.get_level_values(0)[1:].drop_duplicates())
MA_TPEx.index.name ='Moving Average'
for i in range(0,len(MA_TWSE.columns)):
    for j in range(0,len(MA_TWSE)):
        MA_TWSE.iloc[j,i] = round(((StockPrice_TWSE.iloc[int('-' + MA_TWSE.index.str.extract('(\d+)',expand=False)[j]):,i*4+1] * StockPrice_TWSE.iloc[int('-' + MA_TWSE.index.str.extract('(\d+)',expand=False)[j]):,i*4+2])/(StockPrice_TWSE.iloc[int('-' + MA_TWSE.index.str.extract('(\d+)',expand=False)[j]):,i*4+2].mean())).mean(), 2)
    
for i in range(0,len(MA_TPEx.columns)):
    for j in range(0,len(MA_TPEx)):
        MA_TPEx.iloc[j,i] = round(((StockPrice_TPEx.iloc[int('-' + MA_TPEx.index.str.extract('(\d+)',expand=False)[j]):,i*4+1] * StockPrice_TPEx.iloc[int('-' + MA_TPEx.index.str.extract('(\d+)',expand=False)[j]):,i*4+2])/(StockPrice_TPEx.iloc[int('-' + MA_TPEx.index.str.extract('(\d+)',expand=False)[j]):,i*4+2].mean())).mean(), 2)

In [ ]:
MA_TWSE

In [ ]:
MA_TPEx

In [ ]:
temp = [['<7MA', '<14MA', '<30MA', '<60MA', '<90MA', '<180MA', '<360MA', '<720MA', '<1080MA'], ['>3', '>6', '>9']]
Result_TWSE = pd.DataFrame(columns=pd.MultiIndex.from_product(temp))
Result_TPEx = pd.DataFrame(columns=pd.MultiIndex.from_product(temp))
def Judgment(condition):
  global Result_TWSE
  global Result_TPEx
  if (condition == '7MA'):
    x = 0
  elif (condition == '14MA'):
    x = 1
  elif (condition == '30MA'):
    x = 2
  elif (condition == '60MA'):
    x = 3
  elif (condition == '90MA'):
    x = 4
  elif (condition == '180MA'):
    x = 5
  elif (condition == '360MA'):
    x = 6
  elif (condition == '720MA'):
    x = 7
  elif (condition == '1080MA'):
    x = 8
  Pass_0 = []
  Pass_1 = []
  Pass_2 = []
  for i in range(0,len(list(MA_TWSE))):
    if (MA_TWSE.iloc[x,i] > StockPrice_TWSE.iloc[-1,i*4+1]):
        if ((StockPrice_TWSE.iloc[-1,i*4+4] >= 3) and (StockPrice_TWSE.iloc[-1,i*4+4] < 6)):
            Pass_0.append(MA_TWSE.columns[i])
        elif ((StockPrice_TWSE.iloc[-1,i*4+4] >= 6) and (StockPrice_TWSE.iloc[-1,i*4+4] < 9)):
            Pass_1.append(MA_TWSE.columns[i])
        elif ((StockPrice_TWSE.iloc[-1,i*4+4] >= 9)):
            Pass_2.append(MA_TWSE.columns[i])
  Result_TWSE.iloc[:,x*3] = pd.Series(Pass_0)
  Result_TWSE.iloc[:,x*3+1] = pd.Series(Pass_1)
  Result_TWSE.iloc[:,x*3+2] = pd.Series(Pass_2)
  
  Pass_0 = []
  Pass_1 = []
  Pass_2 = []
  for i in range(0,len(list(MA_TPEx))):
    if (MA_TPEx.iloc[x,i] > StockPrice_TPEx.iloc[-1,i*4+1]):
        if ((StockPrice_TPEx.iloc[-1,i*4+4] >= 3) and (StockPrice_TPEx.iloc[-1,i*4+4] < 6)):
            Pass_0.append(MA_TPEx.columns[i])
        elif ((StockPrice_TPEx.iloc[-1,i*4+4] >= 6) and (StockPrice_TPEx.iloc[-1,i*4+4] < 9)):
            Pass_1.append(MA_TPEx.columns[i])
        elif ((StockPrice_TPEx.iloc[-1,i*4+4] >= 9)):
            Pass_2.append(MA_TPEx.columns[i])
  Result_TPEx.iloc[:,x*3] = pd.Series(Pass_0)
  Result_TPEx.iloc[:,x*3+1] = pd.Series(Pass_1)
  Result_TPEx.iloc[:,x*3+2] = pd.Series(Pass_2)

In [ ]:
for i in range(0,len(MA_TWSE.index)):
    Judgment(MA_TWSE.index[i])
for i in range(0,len(MA_TPEx.index)):
    Judgment(MA_TPEx.index[i])
#Result_TWSE = Result_TWSE.reindex(sorted(Result_TWSE.columns), axis=1)
#Result_TPEx = Result_TPEx.reindex(sorted(Result_TPEx.columns), axis=1)

In [ ]:
Result_TWSE

In [ ]:
Result_TPEx

In [ ]:
Result_TWSE.to_excel('Result_TWSE.xlsx')
Result_TPEx.to_excel('Result_TPEx.xlsx')

In [ ]:
def highlight(series):
    return ['color: red' if value > (series.mean()*3)
            else 'color: orange' if value > (series.mean()*2)
            else None for value in series]

In [ ]:
Stock_TWSE_recentvolume = StockPrice_TWSE.xs('Volume', axis = 1, level = 1)[-14:]
columns_volume = list(Stock_TWSE_recentvolume.columns)
Stock_TWSE_recentvolume.style.apply(highlight, subset=columns_volume)